# pylof のテスト（再試行）

<b>Local Outlier Factor algorithm</b>（LOF）のPython実装事例が下記のURLにありました。

https://github.com/damjankuznar/pylof


## (0) 結論

以下の点から、プロダクションには使用できないと判断しております。


- 訓練データに回答のある質問文を与えても、アノマリーと判定されてしまう


- feature が抽出できない質問文を与えると、アノマリーと判定されない


- feature が 102 件抽出される訓練データでテストしたが、１件の予測に30秒前後かかる

## (1) テストデータ／環境準備

マイオペで使用しているテストデータ（learning/tests/engine/fixtures/ 配下のCSVファイル）をベースに動作確認を行います。

動作確認にあたっては、MySQLdb に接続できないため、ローカル環境テスト用の Bot クラスを使用しています。

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_ptna_conversation.csv',
]
temp_path = TestTool.copy_testdata_csv(learning_dir, csv_file_names)
temp_path[0]

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv]


'/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv'

In [3]:
b, _ = os.path.splitext(temp_path[0])
copied_csv_file_paths = b + '.limited.csv'
copied_csv_file_paths

'/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.limited.csv'

In [4]:
import subprocess

cmd = "head -353 " + temp_path[0] + " > " + copied_csv_file_paths
subprocess.call(cmd, shell=True)

0

## (2) TF-IDFベクターの準備

Bot クラス内に組み込まれている __build_training_set_from_csv 関数をバラして実行しています。

In [5]:
'''
    初期設定
    データファイル、エンコードを指定
    内容は、learn.py を参考にしました。    
'''
from learning.core.learn.learning_parameter import LearningParameter
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': None,
    # 'algorithm': LearningParameter.ALGORITHM_NAIVE_BAYES
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    # 'params_for_algorithm': { 'C': 200 }
    'params_for_algorithm': {}
}
learning_parameter = LearningParameter(attr)

bot_id = 7777
csv_file_encoding = 'utf-8'

### (2-1) 訓練データのTF-IDFベクター

In [6]:
'''
    訓練データの生成（内部で TF-IDF 処理を実行）
    
    text_array.py における TF-IDF 処理では、
    以下の通り「する」をストップワード化指定しております
    
    class TextArray:
        :
        def __build_vectorizer(self):
            :
            vectorizer = TfidfVectorizer(use_idf=False, token_pattern=u'(?u)\\b\\w+\\b', stop_words=['する'])
'''
#from learning.core.training_set.training_message_from_csv import TrainingMessageFromCsv
from prototype.modules.training_message_from_csv import TrainingMessageFromCsv
training_set = TrainingMessageFromCsv(bot_id, copied_csv_file_paths, learning_parameter, encoding=csv_file_encoding)
build_training_set_from_csv = training_set.build()

X = build_training_set_from_csv.x
y = build_training_set_from_csv.y

2017/03/30 PM 11:30:57 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 72
2017/03/30 PM 11:30:57 TextArray#__init__ start
2017/03/30 PM 11:30:57 TextArray#to_vec start
2017/03/30 PM 11:30:57 TextArray#to_vec end


In [7]:
n_sample = X.shape[0]
n_feature = X.shape[1]
print("sample=%d, feature=%d" % (n_sample, n_feature))

sample=72, feature=102


### (2-2) 外れデータのTF-IDFベクター

<b><a href="12.ipynb">こちらで再検証した時と同じ質問文</a></b>をつかっております。

In [8]:
'''
    マイオペのプロダクション・コードと同じように、
    訓練データ作成時と同じベクトライザーを使用します。
'''
#from learning.core.training_set.text_array import TextArray
from prototype.modules.text_array import TextArray

test_X = [
    '要素技術は自然languageの機械learningですか？', # まったくfeatureが抽出されない質問文
    '何か習い事をしますか？', # featureが1件抽出される質問文
    '月謝は？', # featureが1件抽出される質問文（訓練データに回答あり）
    '何か習い事がいいですか？', # featureが2件抽出される質問文
    '入会の特典は？', # featureが2件抽出される質問文（訓練データに回答あり）
    'レッスンは何がいいでしょうか？', # featureが3件抽出される質問文
    '特定の音楽大学に勤務している先生を紹介して頂きたいのですが？', # featureが3件抽出される質問文（訓練データに回答あり）

]
vectorizer = training_set.body_array.vectorizer
text_array = TextArray(test_X, vectorizer=vectorizer)

'''
    外れデータのTF-IDFベクターを取得
'''
X_error = text_array.to_vec()

2017/03/30 PM 11:30:57 TextArray#__init__ start
2017/03/30 PM 11:30:57 TextArray#to_vec start
2017/03/30 PM 11:30:57 TextArray#to_vec end


In [9]:
'''
    外れデータのボキャブラリーを確認
'''
vocabulary = text_array._vectorizer.vocabulary_
dumped_features = TestTool.get_dumped_features(X_error, vocabulary)
for d in dumped_features:
    print(d)

index=0[]
index=1[何=1.000]
index=2[月謝=1.000]
index=3[いい=0.707 何=0.707]
index=4[入会=0.707 特典=0.707]
index=5[いい=0.577 レッスン=0.577 何=0.577]
index=6[先生=0.577 紹介=0.577 音楽=0.577]


## (3) LOFの計算

In [10]:
'''
    データ形式を array へ変換
'''
arr_train = X.toarray()
arr_error = X_error.toarray()

In [11]:
from prototype.modules.lof import LOF
from time import time

'''
    LOF オブジェクト初期化
'''
print("Initialize LOF instance...")
t0 = time()

lof = LOF(arr_train)

print("done in %0.3fs." % (time() - t0))

Initialize LOF instance...
done in 0.010s.


### 計算結果を表示します。

- feature が１件も抽出されない質問文（index=0）がアノマリーと判定されないです。

- 訓練データに回答がある質問文（index=2, 6) がアノマリーと判定されてしまいます。

In [12]:
'''
    LOF の計算
'''
print("Calculating local outlier factor...")
t0 = time()

for idx, instance in enumerate(arr_error):
    t0 = time()

    value = lof.local_outlier_factor(5, instance)
    if value > 1:
        print('index=%d[%s], value=%f, done in %0.3fs.' % (idx, test_X[idx], value, time() - t0), '---> Detected as anomaly')
    else:
        print('index=%d[%s], value=%f, done in %0.3fs.' % (idx, test_X[idx], value, time() - t0))

Calculating local outlier factor...
index=0[要素技術は自然languageの機械learningですか？], value=0.979327, done in 35.371s.
index=1[何か習い事をしますか？], value=1.149364, done in 33.846s. ---> Detected as anomaly
index=2[月謝は？], value=1.025909, done in 27.745s. ---> Detected as anomaly
index=3[何か習い事がいいですか？], value=1.144345, done in 27.126s. ---> Detected as anomaly
index=4[入会の特典は？], value=0.873980, done in 4.848s.
index=5[レッスンは何がいいでしょうか？], value=1.131074, done in 28.934s. ---> Detected as anomaly
index=6[特定の音楽大学に勤務している先生を紹介して頂きたいのですが？], value=1.359415, done in 29.024s. ---> Detected as anomaly
